## ch3 신경망

In [3]:
# MNIST 데이터셋 : 손글씨 숫자 이미지 집합
# MNIST 데이터셋 다운 -> 이미지 넘파이 배열로 변환

import sys, os
sys.path.append(os.pardir)  # 부모 디렉터리의 파일을 가져올 수 있도록 설정
import numpy as np
from dataset.mnist import load_mnist # dataset의 mnist 파일을 통해 mnist 실행
from PIL import Image # 이미지 로드 모듈

def img_show(img):
    pil_img = Image.fromarray(np.uint8(img))
    pil_img.show()

(x_train, t_train), (x_test, t_test) = load_mnist(flatten=True, normalize=False)
# (훈련이미지, 훈련레이블), (시험이미지, 시험레이블)
# normalize : 입력 이미지의 픽셀 값을 0.0~1.0 사이의 값으로 정규화할지
# flatten : 입력이미지를 평탄하게 1차원 배열로 만들지
# one_hot_label : 원-핫 인코딩 형태로 저장할지

img = x_train[0]
label = t_train[0]
# 레이블 출력
print(label)  # 5
# 이미지 출력
print(img.shape)  # (784,)
img = img.reshape(28, 28)  # 형상을 원래 이미지의 크기로 변형. flatten을 True로 설정했기 때문에
print(img.shape)  # (28, 28)

img_show(img)

Done
Done
Done
Done
Converting train-images-idx3-ubyte.gz to NumPy Array ...
Done
Converting train-labels-idx1-ubyte.gz to NumPy Array ...
Done
Converting t10k-images-idx3-ubyte.gz to NumPy Array ...
Done
Converting t10k-labels-idx1-ubyte.gz to NumPy Array ...
Done
Creating pickle file ...
Done!
5
(784,)
(28, 28)


In [5]:
import numpy as np



In [7]:
# MINSIT 데이터셋을 가지고 추론을 수행하는 신경망 구현
# 입력 뉴런층 784개 (28x28=784의 이미지 크기)
# 은닉층 2개
# 출력 뉴런층 10개 (0~9까지 숫자)

import sys, os
sys.path.append(os.pardir)  # 부모 디렉터리의 파일을 가져올 수 있도록 설정
import numpy as np
import pickle # 로컬에 저장된 파일 읽기
from dataset.mnist import load_mnist

# 소프트맥스 함수 : 분류
def softmax(x):
    if x.ndim == 2:
        x = x.T
        x = x - np.max(x, axis=0)
        y = np.exp(x) / np.sum(np.exp(x), axis=0)
        return y.T 
    x = x - np.max(x) # 오버플로 대책
    return np.exp(x) / np.sum(np.exp(x))

# 시그모이드 함수 : 활성 함수
def sigmoid(x):
    return 1 / (1 + np.exp(-x))    

# 데이터 받는 방법
def get_data():
    (x_train, t_train), (x_test, t_test) = load_mnist(normalize=True, flatten=True, one_hot_label=False)
    return x_test, t_test


# pickle파일인 sample_wight.pkl에 저장된 '학습된 가중치 매개변수 읽기'
# 아마 weight파일을 따로 저장한듯
def init_network(): 
    with open("sample_weight.pkl", 'rb') as f:
        network = pickle.load(f)
    return network


def predict(network, x):
    # 가중치 및 편향 1번 주기
    W1, W2, W3 = network['W1'], network['W2'], network['W3']
    b1, b2, b3 = network['b1'], network['b2'], network['b3']

    a1 = np.dot(x, W1) + b1
    z1 = sigmoid(a1)
    a2 = np.dot(z1, W2) + b2
    z2 = sigmoid(a2)
    a3 = np.dot(z2, W3) + b3
    y = softmax(a3) # 소프트맥스 처리

    return y


x, t = get_data() # mnist 데이터 얻기 (테스트이미지, 테스트레이블)
network = init_network() # 네트워크 생성
accuracy_cnt = 0
for i in range(len(x)): # x에 저장된 이미지를 하나씩 꺼냄
    y = predict(network, x[i]) # 각 레이블의 확률을 np 배열로 변환
    p= np.argmax(y) # 확률이 가장 높은 원소의 인덱스를 가짐
    if p == t[i]: # if 예측 레이블 == 정답 레이블
        accuracy_cnt += 1

print("Accuracy:" + str(float(accuracy_cnt) / len(x)))
# 0.9352 == 93.52% 정답률

Accuracy:0.9352


In [8]:
# 신경망의 배치 처리
import sys, os
sys.path.append(os.pardir)  # 부모 디렉터리의 파일을 가져올 수 있도록 
import numpy as np
import pickle
from dataset.mnist import load_mnist

# 소프트맥스 함수 : 분류
def softmax(x):
    if x.ndim == 2:
        x = x.T
        x = x - np.max(x, axis=0)
        y = np.exp(x) / np.sum(np.exp(x), axis=0)
        return y.T 
    x = x - np.max(x) # 오버플로 대책
    return np.exp(x) / np.sum(np.exp(x))

# 시그모이드 함수 : 활성 함수
def sigmoid(x):
    return 1 / (1 + np.exp(-x))

def get_data():
    (x_train, t_train), (x_test, t_test) = load_mnist(normalize=True, flatten=True, one_hot_label=False)
    return x_test, t_test

def init_network():
    with open("sample_weight.pkl", 'rb') as f:
        network = pickle.load(f)
    return network

def predict(network, x):
    w1, w2, w3 = network['W1'], network['W2'], network['W3']
    b1, b2, b3 = network['b1'], network['b2'], network['b3']

    a1 = np.dot(x, w1) + b1
    z1 = sigmoid(a1)
    a2 = np.dot(z1, w2) + b2
    z2 = sigmoid(a2)
    a3 = np.dot(z2, w3) + b3
    y = softmax(a3)

    return y


x, t = get_data()
network = init_network()

batch_size = 100 # 배치 크기
accuracy_cnt = 0

for i in range(0, len(x), batch_size): # start, end, step
    x_batch = x[i:i+batch_size] # x[0:100], x[100:200], x[200:300]...
    y_batch = predict(network, x_batch) # 예측
    p = np.argmax(y_batch, axis=1) # 최대값의 인덱스 가져옴
    accuracy_cnt += np.sum(p == t[i:i+batch_size])

print("Accuracy:" + str(float(accuracy_cnt) / len(x)))

Accuracy:0.9352
